In [1]:
import time

from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

%matplotlib inline
plt.style.use("fivethirtyeight")
sns.set_context("notebook")

C:\Users\e739218\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Get the data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [3]:
# Preprocess the data
X_train = np.reshape(X_train, (-1, 28 * 28)).astype("float32")
X_test = np.reshape(X_test, (-1, 28 * 28)).astype("float32")

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 784), (60000, 10), (10000, 784), (10000, 10))

In [4]:
# Define some hyperparameters
batch_size = 128
n_train = X_train.shape[0]
n_test = X_test.shape[0]
learning_rate = 0.001
epochs = 30


g = tf.Graph()

with g.as_default():
    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_data = train_data.batch(batch_size)
    test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    test_data = test_data.batch(batch_size)
    iterator = tf.data.Iterator.from_structure(train_data.output_types,
                                               train_data.output_shapes)
    img, label = iterator.get_next()

    train_init = iterator.make_initializer(train_data)
    test_init = iterator.make_initializer(test_data)

    W = tf.get_variable(
        name="weight", initializer=tf.zeros([784, 10], tf.float32))
    b = tf.get_variable(name="bias", initializer=tf.zeros([1, 10], tf.float32))

    logits = tf.add(tf.matmul(img, W), b, name="logits")

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label, name="entropy"), name="loss")

    optimizer = tf.train.AdamOptimizer().minimize(loss)

    y_pred = tf.nn.softmax(logits, name="predictions")
    correct_pred = tf.equal(tf.argmax(label, axis=1),
                            tf.argmax(y_pred, axis=1), name="correct_predictions")
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name="accuracy")

    init = tf.global_variables_initializer()
    
    tf.summary.FileWriter("logdir/", graph=g)
    
with tf.Session(graph=g) as sess:
    sess.run(init)
    costs = []
    
    for epoch in range(epochs):
        sess.run([train_init])
        try:
            while True:
                sess.run([optimizer, loss])
        except tf.errors.OutOfRangeError:
            pass

    test_accuracy = 0   
    sess.run(test_init)
    try:
        while True:
            batch_accuracy = sess.run(accuracy)
            test_accuracy += batch_accuracy
    
    except tf.errors.OutOfRangeError:
        pass
    test_accuracy /= ((n_test // batch_size) + 1)
    
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Test accuracy: 86.30%
